In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sklearn

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense , BatchNormalization
from tensorflow.keras.optimizers import Adam ,RMSprop
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,learning_curve, cross_val_score, KFold
import optuna
from optuna.integration import XGBoostPruningCallback
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler
import warnings
warnings.filterwarnings('ignore')

In [292]:
train_df = pd.read_csv('open/train.csv')

In [293]:
test_df = pd.read_csv('open/test.csv')

In [73]:
y_df = train_df.iloc[:,1]

In [201]:
from sklearn.preprocessing import RobustScaler

In [294]:
train_df['평균'] = train_df.iloc[:,6:95].apply(sum,axis=1)/89

In [295]:
test_df['평균'] = test_df.iloc[:,5:94].apply(sum,axis=1)/89

In [296]:
train_df['기울기']=(train_df.iloc[:,6]-train_df.iloc[:,94])/89

In [297]:
test_df['기울기']=(test_df.iloc[:,5]-test_df.iloc[:,93])/89

In [182]:
train_df['4일차']=train_df.iloc[:,91]-train_df.iloc[:,94]

In [183]:
test_df['4일차']=test_df.iloc[:,90]-test_df.iloc[:,93]

In [184]:
train_df['3일차']=train_df.iloc[:,92]-train_df.iloc[:,94]

In [185]:
test_df['3일차']=test_df.iloc[:,91]-test_df.iloc[:,93]

In [60]:
train_df = train_df.iloc[:,2:]

In [89]:
test_df=test_df.iloc[:,1:]

In [300]:
df1 = train_df.iloc[:,2:11]
ddf1 = test_df.iloc[:,1:10]
df2 = train_df.iloc[:,90:95]
ddf2 = test_df.iloc[:,89:94]
df3 =train_df .iloc[:,-5:]
ddf3 = test_df.iloc[:,-5:]

In [313]:
df = pd.concat([df1,df2,df3],axis=1)

In [314]:
ddf = pd.concat([ddf1,ddf2,ddf3],axis=1)

In [17]:
y_df = pd.DataFrame(y_df)

In [285]:
from sklearn.preprocessing import MinMaxScaler

In [286]:
std = MinMaxScaler()
std.fit(df)
df_s=std.transform(df)
ddf_s=std.transform(ddf)

In [201]:
k_fold_num = 5
kfold = KFold(n_splits=k_fold_num, shuffle=True, random_state=0)

In [199]:
def nmae_cv(model):
    nmae = -cross_val_score(model, df, y_df, cv=kfold, 
                           scoring="neg_mean_absolute_error")
    return nmae

In [202]:
def Mae(y, y_pred):
    mae = mean_absolute_error(y, y_pred)
    return mae


In [ ]:
sampler = TPESampler(seed=10)
cut = int(len(df_s)*0.7)
x_train = df_s[:cut]
x_valid = df_s[cut:]
y_train = y_df[:cut]
y_valid = y_df[cut:]

def objective(trial):
    dtrain = lgb.Dataset(x_train, label=y_train)
    dtest = lgb.Dataset(x_valid, label=y_valid)

    param = {
        'objective': 'regression',
        'metric': 'neg_mean_absolute_error',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'max_depth': trial.suggest_int('max_depth', 5, 16),
        # 'early_stopping_rounds': trial.suggest_int('early_stopping_rounds', 10, 2000),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 512),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-8, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 700, 3000),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }

    model = lgb.LGBMRegressor(**param,n_jobs=-1,random_state=43)
    lgb_model = model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)],verbose=0)
    mae = Mae(y_valid, lgb_model.predict(x_valid))
    return mae
        
study_lgb = optuna.create_study(direction='minimize', sampler=sampler)
study_lgb.optimize(objective, n_trials=300)

In [ ]:
parameters = {'max_depth': [9], 'lambda_l1': [0.024165495433417283], 'lambda_l2': [1.0608031131998932e-05], 
'num_leaves': [53], 'learning_rate': [0.0015347480548875386], 'n_estimators': [1454], 
'feature_fraction': [0.7341967229200798], 'bagging_fraction': [0.5187246300766235], 'bagging_freq': [1],
 'min_child_samples': [57]}
model1 = LGBMRegressor(n_jobs =-1, random_state=43)
lgbm_clf = GridSearchCV(model1 ,param_grid=parameters ,cv=kfold, scoring = "neg_root_mean_squared_error" )
lgbm_clf.fit(df_s,y_df)

In [341]:
from catboost import CatBoostRegressor

cb_clf = CatBoostRegressor(
    iterations=17000,
    random_seed=43,
    learning_rate=0.55,
    loss_function='MAE'
)
cb_clf.fit(
    df, y_df) 

0:	learn: 98.5242838	total: 5.77ms	remaining: 1m 38s
1:	learn: 58.7983250	total: 10.4ms	remaining: 1m 28s
2:	learn: 44.8043647	total: 14.1ms	remaining: 1m 19s
3:	learn: 38.2929079	total: 19.5ms	remaining: 1m 23s
4:	learn: 34.0971390	total: 23.7ms	remaining: 1m 20s
5:	learn: 33.6803147	total: 26.4ms	remaining: 1m 14s
6:	learn: 33.0216149	total: 29.3ms	remaining: 1m 11s
7:	learn: 32.3771568	total: 33.3ms	remaining: 1m 10s
8:	learn: 32.0678850	total: 38.4ms	remaining: 1m 12s
9:	learn: 31.3299239	total: 41.5ms	remaining: 1m 10s
10:	learn: 30.8981477	total: 44.5ms	remaining: 1m 8s
11:	learn: 30.7529778	total: 47.3ms	remaining: 1m 7s
12:	learn: 30.3386093	total: 52ms	remaining: 1m 7s
13:	learn: 30.0358242	total: 55.6ms	remaining: 1m 7s
14:	learn: 29.5967661	total: 88.4ms	remaining: 1m 40s
15:	learn: 29.2505538	total: 102ms	remaining: 1m 48s
16:	learn: 29.0009928	total: 147ms	remaining: 2m 27s
17:	learn: 28.8218259	total: 152ms	remaining: 2m 23s
18:	learn: 28.6530976	total: 155ms	remaining: 2

In [289]:
pred1 = lgbm_clf.predict(ddf_s)

In [342]:
pred4 = cb_clf.predict(ddf)

In [319]:
sample = pd.read_csv('open/sample_submission.csv')

In [344]:
sample['착과량(int)'] = pred1

In [345]:
sample.to_csv('마지막잎새.csv',index=False)